# 堅牢性の基本

このノートブックでは、堅牢性の基本的な操作とベストプラクティスを学びます。


In [ ]:
# 例外処理でエラーを適切にハンドリングする

import logging
import time
from typing import Optional

# 悪い例（例外を無視）
def divide_numbers_bad(a, b):
    """数値を割り算（例外を無視）"""
    return a / b

# 良い例（適切な例外処理）
def divide_numbers_good(a, b):
    """数値を割り算（適切な例外処理）"""
    try:
        result = a / b
        return result
    except ZeroDivisionError:
        logging.error(f"ゼロ除算エラー: {a} / {b}")
        return None
    except TypeError as e:
        logging.error(f"型エラー: {e}")
        return None
    except Exception as e:
        logging.error(f"予期しないエラー: {e}")
        return None

# 使用例
print("=== 基本的な例外処理 ===")
print(f"正常な計算: {divide_numbers_good(10, 2)}")
print(f"ゼロ除算: {divide_numbers_good(10, 0)}")
print(f"型エラー: {divide_numbers_good(10, 'a')}")

# ファイル操作での例外処理
def read_file_safely(filename: str) -> Optional[str]:
    """ファイルを安全に読み込み"""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        logging.error(f"ファイルが見つかりません: {filename}")
        return None
    except PermissionError:
        logging.error(f"ファイルの読み込み権限がありません: {filename}")
        return None
    except UnicodeDecodeError:
        logging.error(f"ファイルの文字エンコーディングエラー: {filename}")
        return None
    except Exception as e:
        logging.error(f"ファイル読み込みエラー: {e}")
        return None

# 使用例
content = read_file_safely("nonexistent.txt")
if content is not None:
    print(f"ファイル内容: {content}")
else:
    print("ファイルの読み込みに失敗しました")

# データベース操作での例外処理
import sqlite3
from contextlib import contextmanager

class DatabaseManager:
    """データベース管理クラス"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.init_database()
    
    def init_database(self):
        """データベースを初期化"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute('''
                    CREATE TABLE IF NOT EXISTS users (
                        id INTEGER PRIMARY KEY,
                        name TEXT NOT NULL,
                        email TEXT UNIQUE NOT NULL,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )
                ''')
                conn.commit()
        except sqlite3.Error as e:
            logging.error(f"データベース初期化エラー: {e}")
            raise
    
    @contextmanager
    def get_connection(self):
        """データベース接続を取得"""
        conn = None
        try:
            conn = sqlite3.connect(self.db_path)
            yield conn
        except sqlite3.Error as e:
            logging.error(f"データベース接続エラー: {e}")
            if conn:
                conn.rollback()
            raise
        finally:
            if conn:
                conn.close()
    
    def insert_user(self, name: str, email: str) -> bool:
        """ユーザーを挿入"""
        try:
            with self.get_connection() as conn:
                cursor = conn.execute(
                    "INSERT INTO users (name, email) VALUES (?, ?)",
                    (name, email)
                )
                conn.commit()
                logging.info(f"ユーザーを挿入しました: {name}")
                return True
        except sqlite3.IntegrityError as e:
            logging.error(f"ユニーク制約エラー: {e}")
            return False
        except sqlite3.Error as e:
            logging.error(f"データベースエラー: {e}")
            return False
        except Exception as e:
            logging.error(f"予期しないエラー: {e}")
            return False
    
    def get_user_by_email(self, email: str) -> Optional[dict]:
        """メールアドレスでユーザーを取得"""
        try:
            with self.get_connection() as conn:
                cursor = conn.execute(
                    "SELECT * FROM users WHERE email = ?",
                    (email,)
                )
                row = cursor.fetchone()
                if row:
                    return {
                        'id': row[0],
                        'name': row[1],
                        'email': row[2],
                        'created_at': row[3]
                    }
                return None
        except sqlite3.Error as e:
            logging.error(f"データベースエラー: {e}")
            return None
        except Exception as e:
            logging.error(f"予期しないエラー: {e}")
            return None

# 使用例
print("\n=== データベース操作での例外処理 ===")
db_manager = DatabaseManager("test.db")

# ユーザーを挿入
success = db_manager.insert_user("Alice", "alice@example.com")
print(f"ユーザー挿入: {'成功' if success else '失敗'}")

# 重複したメールアドレス
success = db_manager.insert_user("Bob", "alice@example.com")
print(f"重複ユーザー挿入: {'成功' if success else '失敗'}")

# ユーザーを取得
user = db_manager.get_user_by_email("alice@example.com")
if user:
    print(f"ユーザー情報: {user}")
else:
    print("ユーザーが見つかりません")


In [ ]:
# ログ記録で問題を追跡する

import logging
import sys
import json
from datetime import datetime
from typing import Dict, Any

# ログ設定
def setup_logging(log_level=logging.INFO, log_file=None):
    """ログ設定を初期化"""
    # ログフォーマット
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
    # ルートロガーを設定
    root_logger = logging.getLogger()
    root_logger.setLevel(log_level)
    
    # 既存のハンドラーをクリア
    for handler in root_logger.handlers[:]:
        root_logger.removeHandler(handler)
    
    # コンソールハンドラー
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    root_logger.addHandler(console_handler)
    
    # ファイルハンドラー（指定された場合）
    if log_file:
        file_handler = logging.FileHandler(log_file, encoding='utf-8')
        file_handler.setFormatter(formatter)
        root_logger.addHandler(file_handler)
    
    return root_logger

# 使用例
logger = setup_logging(log_level=logging.DEBUG, log_file="app.log")

logger.debug("デバッグメッセージ")
logger.info("情報メッセージ")
logger.warning("警告メッセージ")
logger.error("エラーメッセージ")
logger.critical("致命的エラーメッセージ")

# 構造化ログ記録
class StructuredLogger:
    """構造化ログ記録クラス"""
    
    def __init__(self, name: str):
        self.logger = logging.getLogger(name)
    
    def log_event(self, level: str, message: str, **kwargs):
        """構造化ログを記録"""
        log_data = {
            'timestamp': datetime.now().isoformat(),
            'level': level,
            'message': message,
            **kwargs
        }
        
        if level.upper() == 'DEBUG':
            self.logger.debug(json.dumps(log_data, ensure_ascii=False))
        elif level.upper() == 'INFO':
            self.logger.info(json.dumps(log_data, ensure_ascii=False))
        elif level.upper() == 'WARNING':
            self.logger.warning(json.dumps(log_data, ensure_ascii=False))
        elif level.upper() == 'ERROR':
            self.logger.error(json.dumps(log_data, ensure_ascii=False))
        elif level.upper() == 'CRITICAL':
            self.logger.critical(json.dumps(log_data, ensure_ascii=False))
    
    def log_user_action(self, user_id: str, action: str, **kwargs):
        """ユーザーアクションを記録"""
        self.log_event('INFO', f"User action: {action}", 
                      user_id=user_id, action=action, **kwargs)
    
    def log_api_request(self, method: str, url: str, status_code: int, 
                       response_time: float, **kwargs):
        """APIリクエストを記録"""
        self.log_event('INFO', f"API request: {method} {url}", 
                      method=method, url=url, status_code=status_code, 
                      response_time=response_time, **kwargs)
    
    def log_error(self, error: Exception, context: str = "", **kwargs):
        """エラーを記録"""
        self.log_event('ERROR', f"Error in {context}: {str(error)}", 
                      error_type=type(error).__name__, 
                      error_message=str(error), context=context, **kwargs)

# 使用例
print("\n=== 構造化ログ記録 ===")
structured_logger = StructuredLogger("app")

# ユーザーアクションを記録
structured_logger.log_user_action("user123", "login", ip_address="192.168.1.1")

# APIリクエストを記録
structured_logger.log_api_request("GET", "/api/users", 200, 0.15, user_id="user123")

# エラーを記録
try:
    result = 1 / 0
except ZeroDivisionError as e:
    structured_logger.log_error(e, "calculation", operation="division")

# アプリケーションログ管理
class ApplicationLogger:
    """アプリケーションログ管理クラス"""
    
    def __init__(self, app_name: str):
        self.app_name = app_name
        self.logger = logging.getLogger(app_name)
        self.setup_logging()
    
    def setup_logging(self):
        """ログ設定を初期化"""
        # ログフォーマット
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
        )
        
        # コンソールハンドラー
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)
        
        # ファイルハンドラー
        file_handler = logging.FileHandler(f"{self.app_name}.log", encoding='utf-8')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        
        # エラーファイルハンドラー
        error_handler = logging.FileHandler(f"{self.app_name}_error.log", encoding='utf-8')
        error_handler.setLevel(logging.ERROR)
        error_handler.setFormatter(formatter)
        self.logger.addHandler(error_handler)
    
    def log_startup(self, version: str, config: Dict[str, Any]):
        """アプリケーション起動を記録"""
        self.logger.info(f"Application started", version=version, config=config)
    
    def log_shutdown(self, reason: str = "normal"):
        """アプリケーション終了を記録"""
        self.logger.info(f"Application shutdown", reason=reason)
    
    def log_database_operation(self, operation: str, table: str, 
                              success: bool, duration: float):
        """データベース操作を記録"""
        level = 'INFO' if success else 'ERROR'
        self.logger.log(getattr(logging, level), 
                       f"Database operation: {operation}", 
                       operation=operation, table=table, 
                       success=success, duration=duration)
    
    def log_performance(self, operation: str, duration: float, 
                      memory_usage: float = None):
        """パフォーマンスを記録"""
        self.logger.info(f"Performance: {operation}", 
                        operation=operation, duration=duration, 
                        memory_usage=memory_usage)

# 使用例
print("\n=== アプリケーションログ管理 ===")
app_logger = ApplicationLogger("myapp")

# アプリケーション起動
app_logger.log_startup("1.0.0", {"debug": True, "database": "sqlite"})

# データベース操作
app_logger.log_database_operation("INSERT", "users", True, 0.05)
app_logger.log_database_operation("UPDATE", "users", False, 0.1)

# パフォーマンス
app_logger.log_performance("data_processing", 2.5, 1024.5)

# アプリケーション終了
app_logger.log_shutdown("user_request")


In [ ]:
# バリデーションで入力データを検証する

import re
from typing import Any, Dict, List, Optional, Union
from datetime import datetime

class DataValidator:
    """データバリデーションクラス"""
    
    def __init__(self):
        self.errors = []
    
    def validate_email(self, email: str) -> bool:
        """メールアドレスを検証"""
        if not email:
            self.errors.append("メールアドレスは必須です")
            return False
        
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(pattern, email):
            self.errors.append(f"無効なメールアドレス形式: {email}")
            return False
        
        return True
    
    def validate_phone(self, phone: str) -> bool:
        """電話番号を検証"""
        if not phone:
            self.errors.append("電話番号は必須です")
            return False
        
        # 日本の電話番号形式を検証
        pattern = r'^0\d{1,4}-\d{1,4}-\d{4}$'
        if not re.match(pattern, phone):
            self.errors.append(f"無効な電話番号形式: {phone}")
            return False
        
        return True
    
    def validate_age(self, age: int) -> bool:
        """年齢を検証"""
        if not isinstance(age, int):
            self.errors.append("年齢は整数である必要があります")
            return False
        
        if age < 0 or age > 150:
            self.errors.append(f"無効な年齢: {age}")
            return False
        
        return True
    
    def validate_password(self, password: str) -> bool:
        """パスワードを検証"""
        if not password:
            self.errors.append("パスワードは必須です")
            return False
        
        if len(password) < 8:
            self.errors.append("パスワードは8文字以上である必要があります")
            return False
        
        if not re.search(r'[A-Z]', password):
            self.errors.append("パスワードには大文字が含まれている必要があります")
            return False
        
        if not re.search(r'[a-z]', password):
            self.errors.append("パスワードには小文字が含まれている必要があります")
            return False
        
        if not re.search(r'\d', password):
            self.errors.append("パスワードには数字が含まれている必要があります")
            return False
        
        return True
    
    def validate_user_data(self, user_data: Dict[str, Any]) -> bool:
        """ユーザーデータを検証"""
        self.errors = []
        
        # 必須フィールドの検証
        required_fields = ['name', 'email', 'age']
        for field in required_fields:
            if field not in user_data or not user_data[field]:
                self.errors.append(f"{field}は必須です")
        
        # 各フィールドの検証
        if 'email' in user_data:
            self.validate_email(user_data['email'])
        
        if 'age' in user_data:
            self.validate_age(user_data['age'])
        
        if 'phone' in user_data:
            self.validate_phone(user_data['phone'])
        
        if 'password' in user_data:
            self.validate_password(user_data['password'])
        
        return len(self.errors) == 0
    
    def get_errors(self) -> List[str]:
        """エラーリストを取得"""
        return self.errors.copy()

# 使用例
print("=== データバリデーション ===")

validator = DataValidator()

# 有効なユーザーデータ
valid_user = {
    'name': 'Alice',
    'email': 'alice@example.com',
    'age': 25,
    'phone': '090-1234-5678',
    'password': 'Password123'
}

if validator.validate_user_data(valid_user):
    print("ユーザーデータは有効です")
else:
    print(f"バリデーションエラー: {validator.get_errors()}")

# 無効なユーザーデータ
invalid_user = {
    'name': '',
    'email': 'invalid-email',
    'age': -5,
    'phone': '123-456-789',
    'password': 'weak'
}

if validator.validate_user_data(invalid_user):
    print("ユーザーデータは有効です")
else:
    print(f"バリデーションエラー: {validator.get_errors()}")

# 設定管理でのバリデーション
class ConfigValidator:
    """設定バリデーションクラス"""
    
    def __init__(self):
        self.errors = []
    
    def validate_database_url(self, url: str) -> bool:
        """データベースURLを検証"""
        if not url:
            self.errors.append("データベースURLは必須です")
            return False
        
        if not url.startswith(('sqlite://', 'postgresql://', 'mysql://')):
            self.errors.append(f"無効なデータベースURL形式: {url}")
            return False
        
        return True
    
    def validate_port(self, port: Union[int, str]) -> bool:
        """ポート番号を検証"""
        try:
            port_int = int(port)
            if port_int < 1 or port_int > 65535:
                self.errors.append(f"無効なポート番号: {port}")
                return False
        except ValueError:
            self.errors.append(f"ポート番号は整数である必要があります: {port}")
            return False
        
        return True
    
    def validate_log_level(self, level: str) -> bool:
        """ログレベルを検証"""
        valid_levels = ['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL']
        if level.upper() not in valid_levels:
            self.errors.append(f"無効なログレベル: {level}")
            return False
        
        return True
    
    def validate_config(self, config: Dict[str, Any]) -> bool:
        """設定を検証"""
        self.errors = []
        
        # データベースURL
        if 'database_url' in config:
            self.validate_database_url(config['database_url'])
        
        # ポート番号
        if 'port' in config:
            self.validate_port(config['port'])
        
        # ログレベル
        if 'log_level' in config:
            self.validate_log_level(config['log_level'])
        
        return len(self.errors) == 0
    
    def get_errors(self) -> List[str]:
        """エラーリストを取得"""
        return self.errors.copy()

# 使用例
print("\n=== 設定バリデーション ===")

config_validator = ConfigValidator()

# 有効な設定
valid_config = {
    'database_url': 'sqlite:///app.db',
    'port': 8080,
    'log_level': 'INFO'
}

if config_validator.validate_config(valid_config):
    print("設定は有効です")
else:
    print(f"バリデーションエラー: {config_validator.get_errors()}")

# 無効な設定
invalid_config = {
    'database_url': 'invalid://db',
    'port': 99999,
    'log_level': 'INVALID'
}

if config_validator.validate_config(invalid_config):
    print("設定は有効です")
else:
    print(f"バリデーションエラー: {config_validator.get_errors()}")

# APIリクエストのバリデーション
class APIValidator:
    """APIリクエストバリデーションクラス"""
    
    def __init__(self):
        self.errors = []
    
    def validate_pagination(self, page: int, per_page: int) -> bool:
        """ページネーションを検証"""
        if page < 1:
            self.errors.append("ページ番号は1以上である必要があります")
            return False
        
        if per_page < 1 or per_page > 100:
            self.errors.append("1ページあたりの件数は1-100の範囲である必要があります")
            return False
        
        return True
    
    def validate_sort_order(self, sort_by: str, order: str) -> bool:
        """ソート順を検証"""
        valid_fields = ['id', 'name', 'email', 'created_at']
        if sort_by not in valid_fields:
            self.errors.append(f"無効なソートフィールド: {sort_by}")
            return False
        
        if order.upper() not in ['ASC', 'DESC']:
            self.errors.append(f"無効なソート順: {order}")
            return False
        
        return True
    
    def validate_date_range(self, start_date: str, end_date: str) -> bool:
        """日付範囲を検証"""
        try:
            start = datetime.fromisoformat(start_date)
            end = datetime.fromisoformat(end_date)
            
            if start > end:
                self.errors.append("開始日は終了日より前である必要があります")
                return False
        except ValueError:
            self.errors.append("無効な日付形式")
            return False
        
        return True
    
    def validate_api_request(self, request_data: Dict[str, Any]) -> bool:
        """APIリクエストを検証"""
        self.errors = []
        
        # ページネーション
        if 'page' in request_data and 'per_page' in request_data:
            self.validate_pagination(request_data['page'], request_data['per_page'])
        
        # ソート
        if 'sort_by' in request_data and 'order' in request_data:
            self.validate_sort_order(request_data['sort_by'], request_data['order'])
        
        # 日付範囲
        if 'start_date' in request_data and 'end_date' in request_data:
            self.validate_date_range(request_data['start_date'], request_data['end_date'])
        
        return len(self.errors) == 0
    
    def get_errors(self) -> List[str]:
        """エラーリストを取得"""
        return self.errors.copy()

# 使用例
print("\n=== APIリクエストバリデーション ===")

api_validator = APIValidator()

# 有効なリクエスト
valid_request = {
    'page': 1,
    'per_page': 20,
    'sort_by': 'name',
    'order': 'ASC',
    'start_date': '2023-01-01',
    'end_date': '2023-12-31'
}

if api_validator.validate_api_request(valid_request):
    print("APIリクエストは有効です")
else:
    print(f"バリデーションエラー: {api_validator.get_errors()}")

# 無効なリクエスト
invalid_request = {
    'page': 0,
    'per_page': 200,
    'sort_by': 'invalid_field',
    'order': 'INVALID',
    'start_date': '2023-12-31',
    'end_date': '2023-01-01'
}

if api_validator.validate_api_request(invalid_request):
    print("APIリクエストは有効です")
else:
    print(f"バリデーションエラー: {api_validator.get_errors()}")
